In [1]:
%cd /home/burak/repos/smartrag

/home/burak/repos/smartrag


In [5]:
from datasets import load_dataset

import pandas as pd
import wandb
import os

In [ ]:
%%capture
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="smartrag",
    name="ambiguous_query_generation",
    save_code=False,
)  # start a new wandb run to track this script

In [3]:
from disambiguation_methods.generate_abbreviations import generate_ambiguous_queries
from disambiguation_methods.api_suggestion import get_abbreviation_suggestions as get_abbreviation_suggestions_api
from disambiguation_methods.llm_suggestion import get_abbreviation_suggestions as get_abbreviation_suggestions_llm
from disambiguation_methods.ambiguity_extractor import extract_ambiguities
from disambiguation_methods.intent_extractor import extract_intent

In [ ]:
!wget https://storage.googleapis.com/ai2i/strategyqa/data/strategyqa_dataset.zip -O strategyqa.zip
!unzip strategyqa.zip -d dataset
!wget https://nlp.cs.washington.edu/triviaqa/data/triviaqa-unfiltered.tar.gz -O triviaqa.tar.gz
!tar xzvf triviaqa.tar.gz
!wget https://nlp.cs.washington.edu/ambigqa/data/ambignq_light.zip -O ambignq.zip
!unzip ambignq.zip -d dataset

In [4]:
n_queries = 50
top_n = 10
dataset_name = "generated"
domain = None
GENERATE = False

In [6]:
if GENERATE:
    df = generate_ambiguous_queries(n_queries=n_queries, temperature=1.0)
    df.to_csv(f"./data/{dataset_name}_ambiguous.csv")
else:
    if not os.path.exists(f"./data/{dataset_name}_ambiguous.csv"):
        # dataset_strategyqa = pd.DataFrame(json.loads(open("dataset/strategyqa_train.json").read()))
        # dataset_triviaqa = pd.DataFrame(json.loads(open("triviaqa-unfiltered/unfiltered-web-dev.json").read())["Data"])
        # dataset_ambigqa = pd.DataFrame(json.loads(open("/content/dataset/dev_light.json").read()))
        # dataset_squad = pd.DataFrame(load_dataset("rajpurkar/squad")["train"])
        dataset = pd.DataFrame(load_dataset("keivalya/MedQuad-MedicalQnADataset")["train"])
        dataset.rename(columns={"Question": "question", "Answer": "answer"}, inplace=True)
        # with open("dev.jsonl") as f:
        #  dataset_boolq = pd.json_normalize(map(lambda x: json.loads(x), f.readlines()))
        n_sample = 500
        df = dataset.sample(n_sample, random_state=40).reset_index(drop=True)

        extract_ambiguities(df)
        df = df.dropna(axis=0)

        df.to_csv(f"./data/{dataset_name}_ambiguous.csv")

In [14]:
df = pd.read_csv(f"./data/{dataset_name}_ambiguous.csv")

In [17]:
get_abbreviation_suggestions_api(df, top_n, domain)
get_abbreviation_suggestions_llm(df, top_n, domain)
extract_intent(df, top_n, domain)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [03:11<00:00,  1.92s/it]


In [20]:
table = wandb.Table(dataframe=df)
wandb.log({f"{dataset_name}_ambiguous_top{top_n}+LLM+Intent": table})
df.to_csv(f"./data/{dataset_name}_ambiguous_top{top_n}+LLM+Intent.csv", index=False)